### ASTRADB VectorStore
Pasa de la idea de aplicación a producción con la Plataforma de IA con Astra DB, la base de datos de ultra-baja latencia hecha para IA y Langflow, el IDE RAG de bajo código.
https://www.datastax.com/

### ASTRADB VectorStore

In [ ]:
# Instalar las dependencias necesarias para trabajar con AstraDB y LangChain
# langchain: framework principal para construir aplicaciones con LLMs (versión 0.3.x)
# langchain-core: componentes centrales de LangChain (versión 0.3.x)
# langchain-astradb: integración de LangChain con AstraDB para almacenamiento vectorial (versión 0.6.x)
!pip install \
    "langchain>=0.3.23,<0.4" \
    "langchain-core>=0.3.52,<0.4" \
    "langchain-astradb>=0.6,<0.7"

In [ ]:
# Instalar el paquete langchain_openai para usar embeddings de OpenAI
# Este paquete proporciona acceso a los modelos de embeddings de OpenAI
# que convierten texto en vectores numéricos para búsqueda semántica
!pip install langchain_openai

In [ ]:
### Configuración

# URL del endpoint de la API de AstraDB
# Este es el punto de conexión único para tu instancia de base de datos en la nube de DataStax
# IMPORTANTE: Reemplaza esta URL con la de tu propia instancia de AstraDB
ASTRA_DB_API_ENDPOINT = os.getenv("ASTRA_DB_API_ENDPOINT", "https://tu-endpoint.apps.astra.datastax.com")

# Token de autenticación para acceder a AstraDB
# Este token proporciona credenciales de acceso a tu base de datos AstraDB
# IMPORTANTE: Este token debe estar en el archivo .env, NO en el código
ASTRA_DB_APPLICATION_TOKEN = os.getenv("ASTRA_DB_APPLICATION_TOKEN")

In [ ]:
# Importar la clase OpenAIEmbeddings para crear embeddings usando modelos de OpenAI
from langchain_openai import OpenAIEmbeddings

# Crear una instancia del modelo de embeddings de OpenAI
# model: "text-embedding-3-small" es un modelo de embeddings eficiente y económico
# dimensions: 1024 especifica el tamaño del vector resultante (más dimensiones = más precisión pero más costo)
# La API key se obtiene automáticamente de la variable de entorno OPENAI_API_KEY
embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small",
    dimensions=1024
)

In [ ]:
# Importar la clase OpenAIEmbeddings para crear embeddings usando modelos de OpenAI
from langchain_openai import OpenAIEmbeddings

# Crear una instancia del modelo de embeddings de OpenAI
# model: "text-embedding-3-small" es un modelo de embeddings eficiente y económico
# dimensions: 1024 especifica el tamaño del vector resultante (más dimensiones = más precisión pero más costo)
# api_key: clave de API de OpenAI para autenticación (IMPORTANTE: usar variables de entorno en producción)
embeddings=OpenAIEmbeddings(model="text-embedding-3-small",dimensions=1024,)

In [ ]:
# Mostrar la configuración del objeto embeddings
# Esto permite verificar que el modelo se inicializó correctamente con todos sus parámetros
embeddings

In [ ]:
# Importar la clase AstraDBVectorStore para usar AstraDB como almacén de vectores
from langchain_astradb import AstraDBVectorStore

# Crear una instancia del vector store conectado a AstraDB
# embedding: el modelo de embeddings que convertirá textos en vectores
# api_endpoint: URL del endpoint de la API de AstraDB para conectarse a la base de datos
# collection_name: nombre de la colección donde se almacenarán los vectores
# token: token de autenticación para acceder a AstraDB
# namespace: espacio de nombres para organizar las colecciones (None usa el predeterminado)
vector_store=AstraDBVectorStore(
    embedding=embeddings,
    api_endpoint=ASTRA_DB_API_ENDPOINT,
    collection_name="astra_vector_langchain",
    token=ASTRA_DB_APPLICATION_TOKEN,
    namespace=None,

)
# Mostrar el objeto vector_store para verificar la conexión
vector_store

In [ ]:
# Importar la clase Document para crear objetos de documento con contenido y metadatos
from langchain_core.documents import Document

# Crear el primer documento: Tweet sobre desayuno
document_1 = Document(
    page_content="Desayuné panqueques con chispas de chocolate y huevos revueltos esta mañana.",
    metadata={"source": "tweet"},
)

# Crear el segundo documento: Noticia sobre el clima
document_2 = Document(
    page_content="El pronóstico del tiempo para mañana es nublado y cubierto, con una máxima de 62 grados.",
    metadata={"source": "news"},
)

# Crear el tercer documento: Tweet sobre un proyecto con LangChain
document_3 = Document(
    page_content="Construyendo un nuevo proyecto emocionante con LangChain - ¡ven a verlo!",
    metadata={"source": "tweet"},
)

# Crear el cuarto documento: Noticia sobre un robo bancario
document_4 = Document(
    page_content="Los ladrones asaltaron el banco de la ciudad y robaron 1 millón de dólares en efectivo.",
    metadata={"source": "news"},
)

# Crear el quinto documento: Tweet sobre una película
document_5 = Document(
    page_content="¡Guau! Esa fue una película increíble. No puedo esperar para verla de nuevo.",
    metadata={"source": "tweet"},
)

# Crear el sexto documento: Artículo web sobre el iPhone
document_6 = Document(
    page_content="¿Vale la pena el precio del nuevo iPhone? Lee esta reseña para descubrirlo.",
    metadata={"source": "website"},
)

# Crear el séptimo documento: Artículo web sobre futbolistas
document_7 = Document(
    page_content="Los 10 mejores jugadores de fútbol del mundo en este momento.",
    metadata={"source": "website"},
)

# Crear el octavo documento: Tweet sobre LangGraph
document_8 = Document(
    page_content="¡LangGraph es el mejor framework para construir aplicaciones con estado y agentes!",
    metadata={"source": "tweet"},
)

# Crear el noveno documento: Noticia sobre la bolsa de valores
document_9 = Document(
    page_content="El mercado de valores cayó 500 puntos hoy debido a temores de una recesión.",
    metadata={"source": "news"},
)

# Crear el décimo documento: Tweet con sentimiento negativo
document_10 = Document(
    page_content="Tengo el mal presentimiento de que voy a ser eliminado :(",
    metadata={"source": "tweet"},
)

# Crear una lista con todos los documentos para facilitar su procesamiento
documents = [
    document_1,
    document_2,
    document_3,
    document_4,
    document_5,
    document_6,
    document_7,
    document_8,
    document_9,
    document_10,
]
# Mostrar la lista de documentos
documents

In [ ]:
# Agregar todos los documentos al vector store de AstraDB
# Este método convierte cada documento en un vector usando los embeddings de OpenAI
# y los almacena en la colección de AstraDB en la nube
# Retorna una lista de IDs únicos generados para cada documento almacenado
vector_store.add_documents(documents=documents)

In [ ]:
### Búsqueda desde el Vector Store DB

# Realizar una búsqueda de similaridad en el vector store de AstraDB
# La consulta "Cómo está el clima" se convierte en un vector usando embeddings
# Se compara con todos los vectores almacenados y retorna los más similares (por defecto 4 resultados)
vector_store.similarity_search("Cómo está el clima")

In [ ]:
# Realizar una búsqueda de similaridad con filtros
# Busca documentos similares a la consulta sobre LangChain y abstracciones para LLMs
# k=3: limita los resultados a los 3 documentos más similares
# filter={"source": "tweet"}: solo busca en documentos cuya fuente sea "tweet"
# Itera sobre los resultados y muestra el contenido y metadatos de cada documento
results = vector_store.similarity_search(
    "LangChain proporciona abstracciones para facilitar el trabajo con LLMs",
    k=3,
    filter={"source": "tweet"},
)
for res in results:
    print(f'* "{res.page_content}", metadata={res.metadata}')

In [ ]:
# Realizar una búsqueda de similaridad con puntuación de similitud
# similarity_search_with_score retorna tanto los documentos como su puntuación de similitud
# La puntuación indica qué tan similar es cada documento a la consulta (valores más altos = mayor similitud)
# k=3: retorna los 3 documentos más similares
# filter={"source": "tweet"}: filtra solo tweets
# Muestra cada resultado con su puntuación de similitud (SIM) formateada a 2 decimales
results = vector_store.similarity_search_with_score(
    "LangChain proporciona abstracciones para facilitar el trabajo con LLMs",
    k=3,
    filter={"source": "tweet"},
)
for res, score in results:
    print(f'* [SIM={score:.2f}] "{res.page_content}", metadata={res.metadata}')

In [ ]:
### Retriever (Recuperador)

# Convertir el vector store en un retriever con búsqueda por umbral de similitud
# search_type="similarity_score_threshold": solo retorna documentos con similitud >= al umbral
# search_kwargs: parámetros de búsqueda
#   - k=1: retorna máximo 1 documento
#   - score_threshold=0.5: solo retorna documentos con puntuación de similitud >= 0.5
retriever=vector_store.as_retriever(
  search_type="similarity_score_threshold",
    search_kwargs={"k": 1, "score_threshold": 0.5},
)
# Invocar el retriever con una consulta sobre crimen bancario
# filter={"source": "news"}: filtra solo documentos de noticias
retriever.invoke("Robar del banco es un crimen", filter={"source": "news"})

In [ ]:
### Retriever (Recuperador)

# Convertir el vector store en un retriever usando búsqueda MMR (Maximal Marginal Relevance)
# search_type="mmr": MMR equilibra relevancia con diversidad en los resultados
# MMR evita retornar documentos muy similares entre sí, maximizando la cobertura de información
# search_kwargs: 
#   - k=1: retorna máximo 1 documento
retriever=vector_store.as_retriever(
  search_type="mmr",
    search_kwargs={"k": 1},
)
# Invocar el retriever con una consulta sobre crimen bancario
# filter={"source": "news"}: filtra solo documentos de noticias
retriever.invoke("Robar del banco es un crimen", filter={"source": "news"})